In [2]:
import numpy as np
import cv2 as cv
import pandas as pd
from google.colab.patches import cv2_imshow #especially for google colab
import matplotlib.pyplot as plt
from PIL import Image



```
bilateral Filter
```



In [4]:
# exp^(-x^2/2*sigma^2)
#bilateral filter part 1
def bilateral_filter(flash,Ambient):
  """
  Both inputs need to be same input
  """
  interpolation_array=[]
  lamb=0.01
  sigma_radial=100
  min_I=np.min(flash)-lamb
  max_I=np.max(flash)+lamb
  segments=int(np.ceil((max_I-min_I)/sigma_radial))
  interpolation_array=[0 for i in range(segments)]
  for j in range(segments):
    Ij=min_I+j*((max_I-min_I)/segments)
    #print(Ij)
    intensity_applied=np.exp(-((flash-Ij)*(flash-Ij)/(sigma_radial**2)))
    #normalization=cv.GaussianBlur(intensity_applied,(3,3),0)
    normalize=cv.GaussianBlur(intensity_applied,(5,5),0)+lamb
    Hj=intensity_applied*Ambient
    final=cv.GaussianBlur(Hj,(5,5),0)
    then=final/normalize
    interpolation_array[j]=then

  return np.array(interpolation_array),np.array([min_I+i*((max_I-min_I)/segments) for i in range(segments)])


In [3]:
#bilateral filter part 2
def interpolation(filtered,X,Ambient):
  length=Ambient.shape[0]
  width=Ambient.shape[1]
  ret=np.zeros((length,width))
  for i in range(length):
    for j in range(width):
      ret[i][j]=np.interp(Ambient[i,j],X,filtered[:,i,j])

  return ret

In [4]:
#t=bilateral_filter(img2[:,:,0],img2[:,:,0])
#goutput=interpolation(t[0],t[1],img2[:,:,0])



```
X-Y Separable bilateral filter
```



In [5]:

# helper for "X-Y_bilateral" function .
def apply_bilateral(data,sigmar,sigmad):
  # we can specify sigmar for spatial gaussian standard deviation,
  # for now we are not using sigmar(spatial standard deviation) ,
  #just with setting value 0 in cv.getGaussiankernel , we are good with it
  center=data[len(data)//2]
  k=1/(np.sqrt(2*3.14)*sigmad)

  bilat_arr=np.zeros(len(data))
  # np.exp(-2t*t/r*r)
  gauss_arr=cv.getGaussianKernel(len(data),0).T
  #print(gauss_arr)
  for i in range(len(data)):

    bilat_arr[i]=np.exp(-1*(2*((data[i]-center)**2))/(2*sigmad*sigmad))

  #print(bilat_arr)
  bilat_arr*=k

  sum=np.sum(bilat_arr*gauss_arr)
  op_data=data*bilat_arr
  op_data=op_data*gauss_arr


  return np.sum(op_data)/sum



In [ ]:
"""def bilateral1d(data,gaussize,sigmad):
  filt=np.zeros(data.shape)
  padd_length=gaussize//2
  paddata=np.pad(data,(padd_length,padd_length),constant_values=0)*1.0
  k=1/(np.sqrt(2*3.14)*sigmad)
  gauss=cv.getGaussianKernel(gaussize,0).T
  #gauss size in odd size.
  for i in range(padd_length,len(data)+padd_length):
    crop=paddata[i-padd_length:i+padd_length+1]
    center=crop[padd_length]
    bilat_arr=np.zeros(len(crop))
    for j in range(len(crop)):
      bilat_arr[j]=np.exp(-1*(2*((crop[j]-center)**2))/(2*sigmad*sigmad))
    bilat_arr*=k
    #print(bilat_arr,gauss)
    sum=np.sum(bilat_arr*gauss)
    op_data=crop*bilat_arr
    op_data=op_data*gauss
    op_data=np.sum(op_data)/sum
    filt[i-padd_length]=op_data

  return filt"""

# we can use apply bilateral function to calculate bilateral operation .
# for any kind of input data structure , we can scrap out data , and apply above function.

In [8]:
def X_Y_bilateral(image,sigmar,sigmad,filtersize):
  """
  @params image :2D np array with float datatype
  @params sigmar: std dev for pixel location difference
  @params sigmad: std dev for pixel magnitude closeness
  @params filtersize: size of the filter (1D) 'should be an odd number'
  """

  x_filter=np.zeros(image.shape)
  bi_img=np.zeros(image.shape)
  padd_length=(filtersize)//2 # integer value
  padded_img=np.pad(image,((padd_length,padd_length),(padd_length,padd_length)),constant_values=0)*1.0 # to convert image into floating datatype
  length,width=padded_img.shape
  #print(padd_length,length)


  #go in x direction  on raw image .
  for i in range(padd_length,length-padd_length):
    for j in range(padd_length,length-padd_length):
        # apply filter
        #print(i,j)

        data=padded_img[i][j-padd_length:j+padd_length+1]
        value=apply_bilateral(data,sigmar,sigmad)
        x_filter[i-2][j-2]=value

  transpose=x_filter.T
  # go in y direction  on the x_filterd image
  padded_img=np.pad(transpose,((2,2),(2,2)),constant_values=0)*1.0
  #print(padded_img.shape)
  for i in range(padd_length,length-padd_length):
    for j in range(padd_length,length-padd_length):
        # apply filter
        #print(i,j)
        data=padded_img[i][j-padd_length:j+padd_length+1]
        value=apply_bilateral(data,sigmar,sigmad)
        bi_img[i-2][j-2]=value


  return bi_img.T

In [1]:
#img=cv.imread('/content/pic14.jpg')
#img=cv.resize(cv.cvtColor(img, cv.COLOR_BGR2GRAY),(300,300))

In [ ]:
#used for creating stacks of processed outputs
"""stack=[]
ref=img
stack.append(img)
for i in range(4):
  get=X_Y_bilateral(ref,7,10,7)
  #name="girlface"+str(i)+".jpg"
  stack.append(get)
  #cv.imwrite(name,get)
  #print(name)
  print("iteration",i+1)
  cv2_imshow(get)
  ref=get
"""

"""
stacked=np.zeros((300,900))
for i in range(3):
  j=i*2+1
  print(j)
  stacked[:,300*i:300*(i+1)]=stack[j-1]
  #img-2nd iter - 4th iter"""



```
Quantization for Grayscale image or 2D images.
```



In [6]:
def Quantize(image,levels):
  """
  @params image : 2D numpy array.
  @params levels: no. of Quantization levels.
  """
  band=256//levels
  print(band)
  float_image=image*1.0
  image_level=float_image//band
  quant_img=image_level*band

  #what if
  #rather than simply forcing them to one of extrema of the interval, we can average the ones in a interval.\
  # lets see.
  """distribution=[[] for i in range(levels)]
  average=[[] for i in range(levels) ]
  length,width=image.shape
  for i in range(length):
    for j in range(width):
      distribution[int(image_level[i][j])].append(image[i][j])

  # we got the image values as per the bucket.
  for i in range(levels):
    average[i]=np.average(np.array(distribution[i]))

  avgquant_img=np.zeros(image.shape)

  for i  in range(length):
    for j in range(width):
      avgquant_img[i][j]=average[int(image_level[i][j])]"""



  return quant_img #,image_level

In [203]:
"""
quant_stacked=np.zeros((300,900))
for i in range(3):
  get=stacked[:,300*i:300*(i+1)]
  get=Quantize(get,8)
  quant_stacked[:,300 *i:300*(i+1)]=get
  cv2_imshow(get)
"""




```
Quantization for colored images, since each pixel is having multidimensional information.
```



In [10]:
#some changes need in calculation of new_centers after first iteration (note)
def kmeansQuantize(Image,levels=8,iteration=100,criteria=1e-4): #for colored images
  pixels=Image.reshape(-1,3) # np array of (pixels,3) shape
  init_centers=pixels[np.random.choice(len(pixels),levels)] # initialize clusters sample
  start=init_centers
  for i in range(iteration):
    euclid_dis=np.linalg.norm(pixels[:,np.newaxis,:]-start,axis=2)  # find distance 1
    clusters_ind=np.argmin(euclid_dis,axis=1)  # allocate to minimum 2
    new_centers=np.array([ pixels[clusters_ind==i].mean(axis=0) for i in range(levels)]) # find new centers  3
    Quantized_pix=new_centers[clusters_ind]
    if np.linalg.norm(new_centers-start)<=criteria:
      break

    start=new_centers
    Quant_img=Quantized_pix.reshape(300, 300, -1).astype(np.uint8)

  return  Quant_img


In [70]:
#img2= cv.imread('/content/pic7.jpg')
#img2=cv.resize(img2,(300,300))

In [ ]:
"""stacked=np.zeros((300,900,3))
#lets try  with colorful images
refb=img2[:,:,0]
refg=img2[:,:,1]
refr=img2[:,:,2]
stacked[:,:300,:]=img2
for i in range(4):
  getb=X_Y_bilateral(refb,7,10,7)
  getg=X_Y_bilateral(refg,7,10,7)
  getr=X_Y_bilateral(refr,7,10,7)
  print("iteration",i+1)
  final=np.zeros((300,300,3))
  final[:,:,0],refb=getb,getb
  final[:,:,1],refg=getg,getg
  final[:,:,2],refr=getr,getr
  if i==1:
    stacked[:,300:600,:]=final
  if i==3:
    stacked[:,600:900,:]=final
  cv2_imshow(final) """

"""
quant_stacked=np.zeros((300,900,3))
for i in range(3):
  get=stacked[:,300*i:300*(i+1),:]
  get=kmeansQuantize(get,8)
  quant_stacked[:,300*i:300*(i+1),:]=get
  cv2_imshow(get)
"""
